<a href="https://colab.research.google.com/github/kteppris/RealWaste/blob/main/Spielwiese_Flemming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Daten laden

In [24]:
import sys
import os
from pathlib import Path

# Funktion, die prüft, ob dieses Skript in Colab ausgeführt wird
def in_colab():
    # sys beinhaltet alle bereits geladenen Pakete
    return 'google.colab' in sys.modules  # google.colab wird automatisch in Colab vorgeladen, daher Indiz für Colab Environment

code_repo_path = Path("/content/drive/MyDrive/MADS/RealWaste") if in_colab() else Path.cwd()

if in_colab():
    # Google Drive einbinden zum persistieren von Code, Daten und Ergebnissen
    from google.colab import drive
    drive.mount('/content/drive')
    if not code_repo_path.exists():
        raise FileNotFoundError("Code und Datenordner konnte nicht gefunden werden. Bitte zuerst das Notebook '1_Hauptnotebook.ipynb' ausführen.")
    else:
        # wechsle Arbeitsverzeichnis zu code_repo_path
        %cd {code_repo_path}
        print(f"Wechsel Arbeitsverzeichnis zu {Path.cwd()}")
        %pip install -r requirements.txt
else:
    print("Stellen Sie sicher, dass das richtige Venv als Kernel ausgewählt ist,\nwelches wie in '1_Hauptnotebook.ipynb' beschrieben erstellt wurde.")

data_path = code_repo_path / "data"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/MADS/RealWaste
Wechsel Arbeitsverzeichnis zu /content/drive/MyDrive/MADS/RealWaste


In [25]:
import os
from PIL import Image
import numpy as np

def load_and_resize_images(data_path, new_size): # Bilder neu skaliert mit dem Argument 'new_size(,)'
    images_resized = {} # Dictionary erstellen
    class_names = [d for d in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, d)) and not d.startswith('.')]  # Versteckte Ordner überspringen

    for class_name in class_names:
        class_folder = os.path.join(data_path, class_name)
        image_files = [f for f in os.listdir(class_folder) if f.endswith('.jpg') and not f.startswith('.')]  # Nur .jpg Bilder verarbeiten
        images_resized[class_name] = []

        for image_file in image_files:
            img_path = os.path.join(class_folder, image_file)
            img = Image.open(img_path)
            img_resized = img.resize(new_size)
            images_resized[class_name].append(np.array(img_resized))

    return images_resized

In [ ]:
# Neue Größe auf die Bilder aus meinem Ordner anwenden
images_resized = load_and_resize_images(data_path=data_path, new_size = (128, 128))

# Feature-Engineering

## Farbfeatures: Farbhistogramme

In [26]:
# Dieser Code extrahiert das Farbhistogramm in normalisierter Form als Feature für jedes Bild aus den vorverarbeiteten Daten (images_resized)

def extract_color_features(images, bins=256, channel_range=(0, 256)):
    # Initialisiere eine Liste für Features und eine für Labels
    feature_list = []
    labels_list = []

    # Loope durch die Klassen des Datensets
    for class_name, imgs in images.items():
        # verarbeite jedes Bild in den Klassen
        for img in imgs:
            # Berechne ein Histogramm für jeden Farbkanal (RBG)
            hist_features_color = []
            for i in range(3):  # RGB
                hist = np.histogram(img[:, :, i], bins=bins, range=channel_range)[0]
                hist = hist / hist.sum()  # Histogramm normalisieren
                hist_features_color.extend(hist)

            # Die o.g. Listen feature_list und labels_list füllen
            feature_list.append(hist_features_color)
            labels_list.append(class_name)

    # Konvertieren der Listen in Numpy-Arrays
    feature_array_colorhist = np.array(feature_list)
    label_array_colorhist = np.array(labels_list)

    return feature_array_colorhist, label_array_colorhist

In [ ]:
# Die nachfolgende Zeile extrahiert die features (--> Farbhistogramm) und labels aus den geladenen Bildern
feature_array_colorhist, label_array_colorhist = extract_color_features(images_resized)

## Kantenfeatures: Sobel / Edge Detection als Feature extrahieren

In [ ]:
from scipy.ndimage import sobel
from skimage.color import rgb2gray
import numpy as np

# Annahme: 'images_resized' ist ein Dictionary, das geladene Bilder enthält.
# Dieses Beispiel verwendet ein Dummy-Array für die Demonstration.
# Beispielbild in Graustufen umwandeln (für das echte Bild ersetzen)
# img_gray = rgb2gray(images_resized['class_name'][0])

# Funktion, um die Sobel-Features für ein Bild zu extrahieren
def extract_sobel_features(images):
    sobel_features = []
    for class_images in images.values():
        for img in class_images:
            # Umwandlung in Graustufen, falls das Bild farbig ist
            if len(img.shape) == 3:
                img = rgb2gray(img)

            # Anwenden des Sobel-Filters
            sobel_horizontal = sobel(img, axis=0)  # horizontaler Kantendetektor
            sobel_vertical = sobel(img, axis=1)  # vertikaler Kantendetektor

            # Kombination der horizontalen und vertikalen Kanten zu einem Bild
            sobel_combined = np.hypot(sobel_horizontal, sobel_vertical)

            # Hier könnten weitere Verarbeitungsschritte erfolgen,
            # z.B. Umwandlung in ein flaches Array, wenn notwendig.
            # In diesem Beispiel verwenden wir das kombinierte Sobel-Bild direkt.
            sobel_features.append(sobel_combined.flatten())

    return np.array(sobel_features)


In [27]:
# Extraktion des Sobel-Feature Arrays
feature_array_sobel = extract_sobel_features(images_resized)

## Texturfeatures: Local Binary Patterns (LBP)

In [37]:
from skimage.feature import local_binary_pattern
from skimage.color import rgb2gray
import numpy as np

# Parameter für LBP
P = 8  # Anzahl der kreisförmigen Nachbarn
R = 1  # Radius

# Funktion, um LBP-Features für alle Bilder zu extrahieren
def extract_lbp_features(images_resized, P, R):
    lbp_features = []
    for class_images in images_resized.values():
        for img in class_images:
            # Umwandlung in Graustufen
            img_gray = rgb2gray(img)
            # Berechnung des LBP
            lbp = local_binary_pattern(img_gray, P, R, method="uniform")
            # Optional: Umwandlung des LBP-Bildes in ein Histogramm zur Feature-Reduktion
            # und zur Erstellung eines festen Größenvektors pro Bild
            (hist, _) = np.histogram(lbp.ravel(),
                                     bins=np.arange(0, P + 3),
                                     range=(0, P + 2))
            # Normalisierung des Histogramms
            hist = hist.astype("float")
            hist /= (hist.sum() + 1e-6)
            lbp_features.append(hist)
    return np.array(lbp_features)


In [38]:
# Anwendung der lbp-Funktion auf meine Bilder
feature_array_lbp = extract_lbp_features(images_resized, P, R)

# Standardisierung der Features (Z-Transformation)

In [28]:
# Features normalisieren (Z-Score-Normalisierung)
# Funktion zur Durchführung der Z-Score-Normalisierung
# z=\frac{x-\mu}{\sigma}

import numpy as np

def standardize_features(feature_array):
    # Berechne den Mittelwert und die Standardabweichung für jedes Feature
    mean = np.mean(feature_array, axis=0)
    std = np.std(feature_array, axis=0)

    # Vermeide Division durch Null, indem man dort, wo std Null ist, Einsen setzt
    std[std == 0] = 1

    # Führe die Z-Score-Normalisierung durch
    standardized_array = (feature_array - mean) / std

    return standardized_array

In [29]:
# Standardisierungsfunktion auf das Farbfeature anwenden
standardized_feature_array_colorhist = standardize_features(feature_array_colorhist)

In [30]:
# Standardisierungsfunktion für den Sobel-Array anwenden

standardized_feature_array_edges = standardize_features(feature_array_sobel)

In [39]:
# Standardisierungsfunktion für die Local-Binary-Patterns

standardized_feature_array_lbp = standardize_features(feature_array_lbp)

# Featurekombination (für die extrahierten, standardisierten Features)

In [51]:
# Angenommen, standardized_feature_array_1 und standardized_feature_array_2 sind deine Feature-Arrays
combined_feature_array = np.hstack((standardized_feature_array_colorhist, standardized_feature_array_edges, standardized_feature_array_lbp))

# Modelle trainieren

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Angenommen standardized_feature_array und label_array_colorhist sind bereits definiert.

# 1. Train-Test-Split
X_train, X_test, y_train, y_test = train_test_split(combined_feature_array, label_array_colorhist, test_size=0.2, random_state=42)



In [58]:

# 2. KNN-Modell erstellen
knn = KNeighborsClassifier(n_neighbors=2)  # Du kannst die Anzahl der Nachbarn (n_neighbors) nach Bedarf anpassen

# 3. Modell trainieren
knn.fit(X_train, y_train)

# 4. Modell evaluieren
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Genauigkeit: {accuracy * 100:.2f}%')

Genauigkeit: 32.07%


In [50]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Angenommen standardized_feature_array und label_array_colorhist sind bereits definiert.

# Train-Test-Split
X_train, X_test, y_train, y_test = train_test_split(standardized_feature_array_lbp , label_array_colorhist, test_size=0.2, random_state=42)

# Random Forest Modell erstellen
random_forest = RandomForestClassifier(
    n_estimators=150,
    max_depth=20,
    min_samples_leaf=2,
    min_samples_split=5,
    max_features='sqrt',
    bootstrap=True,
    random_state=42  # Für reproduzierbare Ergebnisse
)  # n_estimators gibt die Anzahl der Bäume im Wald an

# Modell trainieren
random_forest.fit(X_train, y_train)

# Modell evaluieren
y_pred = random_forest.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Genauigkeit: {accuracy * 100:.2f}%')


Genauigkeit: 45.53%


**Gridsearch** für Random Forest zur Hyperparameter-Optimierung verwenden.

In [32]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [33]:
# Parameter-Grid definieren

param_grid = {
    'n_estimators': [200, 300],  # Reduziere auf zwei Möglichkeiten
    'max_depth': [10, 20, None],  # Halte einige Optionen für Variabilität
    'min_samples_leaf': [1, 4],  # Beschränke auf zwei Enden des Spektrums
    'min_samples_split': [2, 10],  # Ebenfalls eine reduzierte Auswahl
    'max_features': ['sqrt'],  # Beschränke dich auf die häufigste Option
    # 'bootstrap': [True]  # Optional: Entferne, wenn du den Standardwert nutzen möchtest
}


In [34]:
# Initialisiere GridSearchCV

# Initialisiere den Random Forest Classifier
rf = RandomForestClassifier()

# Initialisiere GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='accuracy')


In [35]:
# Führe die Grid-Suche aus
grid_search.fit(X_train, y_train)


Fitting 3 folds for each of 24 candidates, totalling 72 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [10, 20, None], 'max_features': ['sqrt'],
                         'min_samples_leaf': [1, 4],
                         'min_samples_split': [2, 10],
                         'n_estimators': [100, 200]},
             scoring='accuracy', verbose=2)

In [36]:
# Ausgabe der besten Parameter
print("Beste Parameter:", grid_search.best_params_)

# Ausgabe der besten Schätzung
best_model = grid_search.best_estimator_

# Verwende das beste Modell für Vorhersagen
predictions = best_model.predict(X_test)

# Berechne die Genauigkeit oder eine andere gewünschte Metrik
from sklearn.metrics import accuracy_score
print("Genauigkeit:", accuracy_score(y_test, predictions))


Beste Parameter: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Genauigkeit: 0.6077812828601472
